In [9]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [10]:
import mne
from mne.io import concatenate_raws

# 5,6,7,10,13,14 are codes for executed and imagined hands/feet
subject_id = 22 # carefully cherry-picked to give nice results on such limited data :)
event_codes = [5,6,9,10,13,14]
#event_codes = [3,4,5,6,7,8,9,10,11,12,13,14]

# This will download the files if you don't have them yet,
# and then return the paths to the files.
physionet_paths = mne.datasets.eegbci.load_data(subject_id, event_codes)

# Load each of the files
parts = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto', verbose='WARNING')
         for path in physionet_paths]

# Concatenate them
raw = concatenate_raws(parts)

# Find the events in this dataset
events, _ = mne.events_from_annotations(raw)

# Use only EEG channels
eeg_channel_inds = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Extract trials, only using EEG channels
epoched = mne.Epochs(raw, events, dict(hands_or_left=2, feet_or_right=3), tmin=1, tmax=4.1, proj=False, picks=eeg_channel_inds,
                baseline=None, preload=True)

Extracting EDF parameters from C:\Users\nemrodov\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S022\S022R05.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from C:\Users\nemrodov\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S022\S022R06.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from C:\Users\nemrodov\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S022\S022R09.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract t

In [13]:
import numpy as np
# Convert data from volt to millivolt
# Pytorch expects float32 for input and int64 for labels.
X = (epoched.get_data() * 1e6).astype(np.float32)
y = (epoched.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1

In [14]:
y

array([0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1], dtype=int64)

In [15]:
from braindecode.datautil.signal_target import SignalAndTarget

train_set = SignalAndTarget(X[:40], y=y[:40])
valid_set = SignalAndTarget(X[40:70], y=y[40:70])


In [16]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = train_set.X.shape[1]
# final_conv_length = auto ensures we only get a single output in the time dimension
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=train_set.X.shape[2],
                        final_conv_length='auto')
if cuda:
    model.cuda()


In [17]:
from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
#optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1,)

In [18]:
model.fit(train_set.X, train_set.y, epochs=30, batch_size=64, scheduler='cosine',
         validation_data=(valid_set.X, valid_set.y),)

2020-01-16 16:30:49,946 INFO : Run until first stop...
2020-01-16 16:30:50,227 INFO : Epoch 0
2020-01-16 16:30:50,228 INFO : train_loss                3.42033
2020-01-16 16:30:50,228 INFO : valid_loss                2.83930
2020-01-16 16:30:50,228 INFO : train_misclass            0.47500
2020-01-16 16:30:50,228 INFO : valid_misclass            0.46667
2020-01-16 16:30:50,229 INFO : runtime                   0.00000
2020-01-16 16:30:50,229 INFO : 
2020-01-16 16:30:50,867 INFO : Time only for training updates: 0.64s
2020-01-16 16:30:51,146 INFO : Epoch 1
2020-01-16 16:30:51,147 INFO : train_loss                0.80030
2020-01-16 16:30:51,147 INFO : valid_loss                1.41157
2020-01-16 16:30:51,147 INFO : train_misclass            0.40000
2020-01-16 16:30:51,148 INFO : valid_misclass            0.50000
2020-01-16 16:30:51,148 INFO : runtime                   0.92002
2020-01-16 16:30:51,148 INFO : 
2020-01-16 16:30:51,768 INFO : Time only for training updates: 0.62s
2020-01-16 16:3

2020-01-16 16:31:05,600 INFO : valid_misclass            0.16667
2020-01-16 16:31:05,601 INFO : runtime                   0.92307
2020-01-16 16:31:05,601 INFO : 
2020-01-16 16:31:06,239 INFO : Time only for training updates: 0.64s
2020-01-16 16:31:06,521 INFO : Epoch 18
2020-01-16 16:31:06,522 INFO : train_loss                0.01759
2020-01-16 16:31:06,522 INFO : valid_loss                0.47952
2020-01-16 16:31:06,522 INFO : train_misclass            0.00000
2020-01-16 16:31:06,523 INFO : valid_misclass            0.16667
2020-01-16 16:31:06,523 INFO : runtime                   0.89065
2020-01-16 16:31:06,523 INFO : 
2020-01-16 16:31:07,150 INFO : Time only for training updates: 0.62s
2020-01-16 16:31:07,445 INFO : Epoch 19
2020-01-16 16:31:07,446 INFO : train_loss                0.01639
2020-01-16 16:31:07,446 INFO : valid_loss                0.48242
2020-01-16 16:31:07,446 INFO : train_misclass            0.00000
2020-01-16 16:31:07,446 INFO : valid_misclass            0.16667
202

In [19]:
model.epochs_df

,train_loss,valid_loss,train_misclass,valid_misclass,runtime
0,3.420332,2.839304,0.475,0.466667,0.000000
1,0.800297,1.411565,0.400,0.500000,0.920017
2,0.364381,0.996504,0.150,0.366667,0.902079
3,0.316833,0.918470,0.150,0.366667,0.926534
4,0.292580,0.842933,0.150,0.366667,0.895620
5,0.307576,0.922559,0.125,0.333333,0.889655
6,0.267159,0.912827,0.125,0.300000,0.907585
7,0.227220,0.850048,0.150,0.333333,0.941027
8,0.183802,0.748607,0.100,0.233333,0.883129
9,0.154119,0.639178,0.050,0.233333,0.913540


In [20]:
test_set = SignalAndTarget(X[70:], y=y[70:])
model.evaluate(test_set.X, test_set.y)

{'loss': 0.3027511239051819, 'misclass': 0.09999999999999998, 'runtime': 0.0}

In [10]:
model.predict_classes(test_set.X)

array([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1],
      dtype=int64)

In [11]:
model.predict_outs(test_set.X)

array([[-2.50973439e+00, -8.47845748e-02],
       [-5.75099364e-02, -2.88441467e+00],
       [-1.46509218e+00, -2.62738019e-01],
       [-1.21271368e-02, -4.41836691e+00],
       [-1.14327455e-02, -4.47698450e+00],
       [-2.40981841e-04, -8.33090973e+00],
       [-8.61561298e+00, -1.81270094e-04],
       [-5.42108975e-02, -2.94185638e+00],
       [-2.17318870e-02, -3.83982086e+00],
       [-2.66500026e-01, -1.45267344e+00],
       [-6.58152938e-01, -7.29410529e-01],
       [-8.12942803e-01, -5.86179554e-01],
       [-1.31112970e-02, -4.34082937e+00],
       [-4.60210514e+00, -1.00813434e-02],
       [-8.17206651e-02, -2.54503036e+00],
       [-1.27797112e-01, -2.12052941e+00],
       [-3.06472588e+00, -4.77906242e-02],
       [-1.12980044e+00, -3.90228361e-01],
       [-7.60704279e-02, -2.61388993e+00],
       [-1.26015663e+00, -3.33529979e-01]], dtype=float32)

In [12]:
import mne
import numpy as np
from mne.io import concatenate_raws
from braindecode.datautil.signal_target import SignalAndTarget

# First 50 subjects as train
physionet_paths = [ mne.datasets.eegbci.load_data(sub_id,[4,8,12,]) for sub_id in range(1,51)]
physionet_paths = np.concatenate(physionet_paths)
parts = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto')
         for path in physionet_paths]

raw = concatenate_raws(parts)

picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Find the events in this dataset
events, _ = mne.events_from_annotations(raw)

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epoched = mne.Epochs(raw, events, dict(hands=2, feet=3), tmin=1, tmax=4.1, proj=False, picks=picks,
                baseline=None, preload=True)

# 51-55 as validation subjects
physionet_paths_valid = [mne.datasets.eegbci.load_data(sub_id,[4,8,12,]) for sub_id in range(51,56)]
physionet_paths_valid = np.concatenate(physionet_paths_valid)
parts_valid = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto')
         for path in physionet_paths_valid]
raw_valid = concatenate_raws(parts_valid)

picks_valid = mne.pick_types(raw_valid.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

events_valid = mne.find_events(raw_valid, shortest_event=0, stim_channel='STI 014')

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epoched_valid = mne.Epochs(raw_valid, events_valid, dict(hands=2, feet=3), tmin=1, tmax=4.1, proj=False, picks=picks_valid,
                baseline=None, preload=True)

train_X = (epoched.get_data() * 1e6).astype(np.float32)
train_y = (epoched.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1
valid_X = (epoched_valid.get_data() * 1e6).astype(np.float32)
valid_y = (epoched_valid.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1
train_set = SignalAndTarget(train_X, y=train_y)
valid_set = SignalAndTarget(valid_X, y=valid_y)

[........................................] 100.00% (  2.5 MB,   1.6 MB/s) \   
[........................................] 100.00% (  2.5 MB,   3.6 MB/s) \   
[........................................] 100.00% (  2.5 MB,  14.8 MB/s) -   
[........................................] 100.00% (  2.4 MB,   3.4 MB/s) -   
[........................................] 100.00% (  2.4 MB,   5.5 MB/s) -   
[........................................] 100.00% (  2.4 MB, 157.9 MB/s) |   
[........................................] 100.00% (  2.5 MB,   5.0 MB/s) \   
[........................................] 100.00% (  2.5 MB,   5.4 MB/s) /   
[........................................] 100.00% (  2.5 MB,   1.6 MB/s) \   
[........................................] 100.00% (  2.4 MB,   4.3 MB/s) |   
[........................................] 100.00% (  2.4 MB,   4.5 MB/s) \   
[........................................] 100.00% (  2.4 MB,   4.3 MB/s) \   
[........................................] 100.00% (

In [8]:
train_set.X.shape

NameError: name 'train_set' is not defined